Author: Jinze Wu

Student Number: p09323028

In [ ]:
import pandas as pd
import datetime as dt

# **Download data**
Download data from google cloud

In [ ]:
!gdown --id '1oTtu7K5NVLCp-RKUuyVtwShLnryIVrKc' --output creditcard.xlsx
!ls

# **Sheet to DataFrame**

In [ ]:
customer = pd.read_excel('creditcard.xlsx', sheet_name='客戶資料檔')
creditcard = pd.read_excel('creditcard.xlsx', sheet_name='信用卡資料檔')
creditlog = pd.read_excel('creditcard.xlsx', sheet_name='信用卡交易記錄檔')
creditlog = creditlog.iloc[:,:8]

In [ ]:
customer.head()

,客戶ID,年齡,生日,居住地,教育程度,性別,婚姻狀況,職業,辦第一張信用卡的時間
0,7293,57,1955-08-24,大台北地區,專科,1:男,已婚,公務員,2006-11-27
1,5963,29,1983-12-05,大台北地區,學士,1:男,單身,白領一般職員,2009-07-24
2,18700,27,1985-12-20,高屏地區,高中,1:男,單身,藍領(勞動工作或作業員)一般職員,2009-09-22
3,920,39,1973-05-29,大台北地區,學士,1:男,已婚,自營事業老闆,2010-03-05
4,4612,26,1986-01-06,嘉南地區,學士,1:男,單身,白領一般職員,2010-08-05


In [ ]:
creditcard.head()

,信用卡ID,客戶ID,信用卡開卡日,信用卡到期日,信用額度,卡等
0,41,2027,2006-05-19,2015-08-31,500000,金卡
1,814,7437,2002-01-14,2016-01-31,200000,普卡
2,905,6475,2002-05-07,2016-05-31,50000,普卡
3,1142,18841,2001-07-23,2015-07-31,250000,金卡
4,1411,14127,2005-10-03,2016-06-30,270000,白金卡


In [ ]:
creditlog.head()

,交易序號,信用卡ID,客戶ID,刷卡日期,刷卡產品產業分類,刷卡類型,刷卡金額,刷卡地點
0,8,6448,15195,2010-12-01,14_旅遊,一般消費,5200,國內
1,9,6826,13687,2010-12-01,12_量販超市,一般消費,250,國內
2,10,6826,13687,2010-12-01,12_量販超市,一般消費,1316,國內
3,11,7896,179,2010-12-01,10_藥妝,一般消費,218,國內
4,20,9387,16756,2010-12-01,11_3C居家電信,一般消費,2610,國內


# **Create RFM Table**

In [ ]:
now = dt.datetime(2012,12,1)
Dates = pd.to_datetime(creditlog['刷卡日期'])
creditlog['刷卡日期'] = Dates

In [ ]:
rfmTable = creditlog.groupby('客戶ID').agg({'刷卡日期': lambda x: (now - x.max()).days, # Recency
                       '客戶ID': lambda x: len(x),      # Frequency
                       '刷卡金額': lambda x: x.mean()}) # Monetary Value

In [ ]:
rfmTable.rename(columns={'刷卡日期': 'Recency',
             '客戶ID': 'Frequency',
             '刷卡金額': 'MV'}, inplace=True)
rfmTable.head()

,Recency,Frequency,MV
客戶ID,,,
89,19,85,1772.717647
106,8,75,1202.560000
131,401,16,4347.375000
179,290,4,2295.500000
261,354,13,1524.230769


In [ ]:
creditlog[creditlog['客戶ID'] == 106]  # check result

,交易序號,信用卡ID,客戶ID,刷卡日期,刷卡產品產業分類,刷卡類型,刷卡金額,刷卡地點
107,413,10745,106,2010-12-10,13_交通(含加值),一般消費,500,國內
265,1021,10745,106,2010-12-24,13_交通(含加值),一般消費,500,國內
497,1951,10745,106,2011-01-14,13_交通(含加值),一般消費,500,國內
652,2606,10745,106,2011-01-28,10_藥妝,一般消費,310,國內
653,2607,10745,106,2011-01-28,13_交通(含加值),一般消費,500,國內
...,...,...,...,...,...,...,...,...
7407,29254,10745,106,2012-10-31,10_藥妝,一般消費,290,國內
7466,29487,10745,106,2012-11-06,12_量販超市,一般消費,509,國內
7573,29911,10745,106,2012-11-16,13_交通(含加值),一般消費,1232,國內
7595,29963,10745,106,2012-11-19,13_交通(含加值),一般消費,1738,國內


# **Gerneral RFM**

## **Determine Quantiles**

In [ ]:
quantiles = rfmTable.quantile(q=[0.2, 0.4, 0.6, 0.8])
quantiles

,Recency,Frequency,MV
0.2,3.0,11.8,963.386994
0.4,5.0,25.2,1453.219444
0.6,22.4,62.2,2247.493333
0.8,113.8,98.4,3545.042594


## **Define Score Function**

In [ ]:
def RClass(x, p, d):
    if x <= d[p][0.2]:
        return 5
    elif x <= d[p][0.4]:
        return 4
    elif x <= d[p][0.6]: 
        return 3
    elif x <= d[p][0.8]: 
        return 2
    else:
        return 1

def FMClass(x,p,d):
    if x <= d[p][0.2]:
        return 1
    elif x <= d[p][0.4]:
        return 2
    elif x <= d[p][0.6]: 
        return 3
    elif x <= d[p][0.8]: 
        return 4
    else:
        return 5


## **Compute RFM**

In [ ]:
rfmTable['RI'] = rfmTable['Recency'].apply(RClass, args=('Recency',quantiles,))
rfmTable['FI'] = rfmTable['Frequency'].apply(FMClass, args=('Frequency',quantiles,))
rfmTable['MI'] = rfmTable['MV'].apply(FMClass, args=('MV',quantiles,))
rfmTable['Score'] = rfmTable['RI'] + rfmTable['FI'] + rfmTable['MI']
rfmTable['RFMclass'] = rfmTable['RI'].map(str) + rfmTable['FI'].map(str) + rfmTable['MI'].map(str)

In [ ]:
rfmTable.head()

,Recency,Frequency,MV,RI,FI,MI,Score,RFMclass
客戶ID,,,,,,,,
89,19,85,1772.717647,3,4,3,10,343
106,8,75,1202.560000,3,4,2,9,342
131,401,16,4347.375000,1,2,5,8,125
179,290,4,2295.500000,1,1,4,6,114
261,354,13,1524.230769,1,2,3,6,123


# **Bob Stone RFM**

## **Define Bob Stone Score Function**

In [ ]:
def BS_RClass(x):
    if x <= 90:
        return 24
    elif x <= 180:
        return 12
    elif x <= 270: 
        return 6
    elif x <= 360: 
        return 3
    else:
        return 0

def BS_MClass(x):
    Mscore = x / 30 * 0.1
    return 9 if Mscore >= 9 else Mscore

## **Compute Bob Stone RFM**

In [ ]:
rfmTable['BS_RI'] = rfmTable['Recency'].apply(BS_RClass)
rfmTable['BS_FI'] = rfmTable['Frequency'].apply(lambda x: 4*x)
rfmTable['BS_MI'] = rfmTable['MV'].apply(BS_MClass)
rfmTable['BS_Score'] = rfmTable['BS_RI'] + rfmTable['BS_FI'] + rfmTable['BS_MI']

# **Result Table**

In [ ]:
rfmTable.shape

(100, 12)

In [ ]:
rfmTable

,Recency,Frequency,MV,RI,FI,MI,Score,RFMclass,BS_RI,BS_FI,BS_MI,BS_Score
客戶ID,,,,,,,,,,,,
89,19,85,1772.717647,3,4,3,10,343,24,340,5.909059,369.909059
106,8,75,1202.560000,3,4,2,9,342,24,300,4.008533,328.008533
131,401,16,4347.375000,1,2,5,8,125,0,64,9.000000,73.000000
179,290,4,2295.500000,1,1,4,6,114,3,16,7.651667,26.651667
261,354,13,1524.230769,1,2,3,6,123,3,52,5.080769,60.080769
...,...,...,...,...,...,...,...,...,...,...,...,...
19239,54,13,3127.230769,2,2,4,8,224,24,52,9.000000,85.000000
19483,1,107,1735.224299,5,5,3,13,553,24,428,5.784081,457.784081
19504,5,98,977.367347,4,4,2,10,442,24,392,3.257891,419.257891
